In [ ]:
import json

import openai
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

client = openai.Client()

**```Explicação Geral sobre o Código e Tools da API da OpenAI```** <br>
 - O código apresentado utiliza a API da OpenAI para gerar respostas baseadas em ferramentas (tools) customizadas. Nesse caso, temos uma função chamada obter_temperatura_atual, que simula a obtenção da temperatura em algumas cidades. O código permite que o modelo da OpenAI utilize essa função como uma ferramenta adicional para fornecer informações ao usuário.

**```Como Funciona a Integração com Ferramentas```**<br>
 - As tools (ferramentas) são integradas ao modelo GPT para permitir que ele execute funções externas e retorne os resultados como parte de sua resposta. Nesse exemplo, o modelo decide qual ferramenta chamar com base nas mensagens do usuário e na descrição das ferramentas disponíveis. O conceito de "tools" amplia as capacidades do modelo, permitindo-lhe fazer cálculos, chamar APIs, ou realizar operações personalizadas.

In [ ]:
def obter_temperatura_atual(local, unidade="celcius"): # a função está hardcoded. Poderia ser uma API.
    if "são paulo" in local.lower():
        return json.dumps(
        {"local": "São Paulo", "temperatura": "32", "unidade": unidade}
        )
    elif "porto alegre" in local.lower():
        return json.dumps(
            {"local": "Porto Alegre", "temperatura": "25", "unidade": unidade}
        )
    elif "rio de janeiro" in local.lower():
        return json.dumps(
            {"local": "Rio de Janeiro", "temperatura": "35", "unidade": unidade}
        )
    else:
        return json.dumps(
            {"local": local, "temperatura": "unknown"}
        ) # json.dumps -> pega um dicionário e transforma em um json.
#---
tools = [ #Temos que descrever para o modelo tudo sobre as Ferramentas/Tools que ele tem para usar!
    {
        "type": "function",
        "function": {
            "name": "obter_temperatura_atual", #importante para o modelo saber a descrição explicita da função!
            "description": "Obtém a temperatura atual em uma dada cidade",
            "parameters": {
                "type": "object", # O tipo dos parâmetros é um objeto JSON
                "properties": {#são os parâmetros da função! def obter_temperatura_atual(local, unidade="celcius")
                    "local": {
                        "type": "string",
                        "description": "O nome da cidade. Ex: São Paulo",
                    },
                    "unidade": {
                        "type": "string",
                        "enum": ["celcius", "fahrenheit"]
                    },
                },
                "required": ["local"],  # O parâmetro "local" é obrigatório
            },
        },
    }
]
#---
funcoes_disponiveis = { # Mapeia o nome das ferramentas para a função real em Python
    "obter_temperatura_atual": obter_temperatura_atual,
}
# Definição da mensagem inicial do usuário solicitando a temperatura em duas cidades
mensagens = [
    {"role": "user",
    "content": "Qual é a temperatura em São Paulo e Porto Alegre?"}
]
# Chama a API da OpenAI para gerar uma resposta
resposta = client.chat.completions.create(
    model="gpt-3.5-turbo-0125",
    messages=mensagens,
    tools=tools, # Define as ferramentas disponíveis para o modelo
    tool_choice="auto", # Permite que o modelo escolha automaticamente qual ferramenta usar
    )
#---
mensagem_resp = resposta.choices[0].message # Obtém a resposta inicial do modelo
tool_calls = mensagem_resp.tool_calls  # Verifica se a resposta inclui chamadas de ferramentas

if tool_calls: # Se houver chamadas de ferramentas, processa cada uma delas
    mensagens.append(mensagem_resp) # Adiciona a resposta inicial à lista de mensagens
    for tool_call in tool_calls:  # Itera sobre cada chamada de ferramenta
        function_name = tool_call.function.name  # Obtém o nome da função chamada
        function_to_call = funcoes_disponiveis[function_name]  # Busca a função correspondente no dicionário de funções
        function_args = json.loads(tool_call.function.arguments)  # Extrai os argumentos da função em formato JSON
        function_response = function_to_call(  # Chama a função real passando os argumentos extraídos
            local=function_args.get("local"),
            unidade=function_args.get("unidade"),
        )
        mensagens.append( # Adiciona a resposta da ferramenta à lista de mensagens
            {
                "tool_call_id": tool_call.id,
                "role": "tool", # A função agindo como uma ferramenta
                "name": function_name,
                "content": function_response,  # Resposta da função (temperatura da cidade)
            }
        )
# Gera uma nova resposta final com base nas mensagens atualizadas (incluindo a resposta da ferramenta)
    segunda_resposta = client.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        messages=mensagens,
    )
# Exibe a resposta final gerada pelo modelo com as informações da ferramenta
mensagens_resp = segunda_resposta.choices[0].message
print(mensagem_resp.content) 

**```Explicação por Blocos```**

**1. Função ```obter_temperatura_atual```:**

 - Essa função simula a obtenção da temperatura de cidades específicas (São Paulo, Porto Alegre, Rio de Janeiro). A função utiliza ``json.dumps`` para retornar os dados no formato JSON.
 - Se o local não for uma dessas cidades, o valor da temperatura será ``"unknown"``, indicando que a cidade não é reconhecida.

**2. Definição de Ferramentas:**
 - Um dicionário com o nome da função e sua descrição detalhada é definido para o modelo, permitindo que ele use essa função quando necessário. O parâmetro obrigatório ``local`` e o opcional ``unidade`` são especificados. Isso orienta o modelo sobre como chamar a função corretamente.

**3. Execução do Modelo com Ferramentas:**

 - O código faz uma chamada para o modelo GPT com uma mensagem do usuário, e permite que o modelo utilize as ferramentas definidas para gerar uma resposta. A chamada de ferramentas é automática com o parâmetro ``"tool_choice": "auto"``.
 - O modelo escolhe qual ferramenta usar com base na descrição da função e nas mensagens do usuário.

**4. Processamento das Chamadas de Ferramentas:**

 - Caso o modelo decida utilizar uma ferramenta, ele retorna os detalhes da chamada. O código então processa essas chamadas, invocando a função Python correspondente e passando os parâmetros.
 - A resposta da função (``obter_temperatura_atual``) é adicionada ao fluxo de mensagens e enviada de volta ao modelo para gerar uma resposta final.
  
**5. Resposta Final:**

 - Após o uso das ferramentas, o modelo gera uma resposta final, incluindo os resultados das funções chamadas. Essa resposta é exibida ao usuário.
---
Esse código demonstra como integrar funções personalizadas como ferramentas dentro da API da OpenAI. O modelo pode chamar funções para realizar tarefas específicas, como calcular ou obter informações, e usa essas respostas para gerar uma interação mais rica com o usuário.